In [ ]:
# Prior to running the code, make sure to download sample_<year>.zip from the Freddie Mac website and unzip them and store them all in the same folder. 
# Provide below the name for the folder where sample_<year> folder are all stored. The output csv file will be copied in this folder. 
 
folder_path = 'Set Your folder path here'

# Now specify the minimum year and maximum year of data you are working with. 

min_year = 1999
max_year = 2022

# At each vintage (quarterly), there are 12500 unique loans. If sample year becomes larger, it quickly becomes computationally infeasible. 
# For that reason, you can specify the number of loans you want to randomly select out of 12500 of the same vintage to work with a subsample.

sample_size = 1000    # If you would like to work with a full dataset, set sample_size = 12500.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Function to read the file and process its contents
def parse_origination(filename, year):
    data = []
    with open(filename, 'r') as file:
        for line in file:
            # Remove spaces and split by '|'
            processed_line = ''.join(line.split()).split('|')
            if len(processed_line) == 32:
                data.append(processed_line)
    
    # Create a DataFrame with 32 columns. Variable definitions and how the data are stored in the txt file is available in Freddie Mac.  
    df = pd.DataFrame(data, columns=['CREDIT_SCORE', 'FIRST_PAYMENT_DATE', 'FIRST_TIME_HOMEBUYER_FLAG', 'MATURITY_DATE', 'METROPOLITAN_DIVISION',
                                     'MORTGAGE_INSURANCE_PERCENTAGE', 'NUMBER_OF_UNITS', 'OCCUPANCY_STATUS', 'ORIGINAL_COMBINED_LOAN-TO-VALUE',
                                     'ORIGINAL_DEBT-TO-INCOME', 'ORIGINAL_UPB', 'ORIGINAL_LOAN-TO-VALUE', 'ORIGINAL_INTEREST_RATE', 'CHANNEL',
                                     'PREPAYMENT_PENALTY_MORTGAGE', 'AMORTIZATION_TYPE', 'PROPERTY_STATE', 'PROPERTY_TYPE', 'POSTAL_CODE',
                                     'LOAN_SEQUENCE_NUMBER', 'LOAN_PURPOSE', 'ORIGINAL_LOAN_TERM', 'NUMBER_OF_BORROWERS', 'SELLER_NAME', 
                                     'SERVICER_NAME', 'SUPER_CONFORMING_FLAG', 'PRE-RELIEF_REFINANCE_LOAN_SEQUENCE_NUMBER', 'PROGRAM_INDICATOR',
                                     'RELIEF_REFINANCE_INDICATOR', 'PROPERTY_VALUATION METHOD', 'INTEREST_ONLY_INDICATOR', 'MI_CANCELLATION_INDICATOR'])
    # Assigns the quarter in which loan is originated to the year. 
    df['VINTAGE'] = str(year) + df['LOAN_SEQUENCE_NUMBER'].str[3:5]
    return df


In [ ]:
# For loop to create a master dataframe that contains all of the origination data. 
origination_data = pd.DataFrame([])
for year in range(min_year, max_year+1):
    filename = f'{folder_path}/sample_{year}/sample_orig_{year}.txt'
    origination_data = pd.concat([origination_data, parse_origination(filename, year)], ignore_index=True)
origination_data.head()

In [ ]:
list = []
for year in range(min_year, max_year+1):
    for i in range(1,5):
        list.append(len(origination_data[origination_data['VINTAGE'] == f'{year}Q{i}']['LOAN_SEQUENCE_NUMBER'].unique()))
pd.DataFrame(list).describe()
# Every vintage has 12500 unique loans

In [ ]:
# At each vintage, randomly select the loan to construct a subsample with the size of 4000 to reduce the sample size (whose size is 12500 at each vintage.)
import random as rand

rand_origination = pd.DataFrame([])
for year in range(min_year, max_year+1):
    for i in range(1,5):
        list = [a for a in origination_data[origination_data['VINTAGE'] == f'{year}Q{i}']['LOAN_SEQUENCE_NUMBER']]
        randomlist = rand.sample(list, sample_size)
        data = pd.DataFrame({'LOAN_SEQUENCE_NUMBER':randomlist})
        data['VINTAGE'] = f'{year}Q{i}'
        rand_origination = pd.concat([rand_origination, data], ignore_index=True)

In [ ]:
# Now that I have a smaller random sample, I can left join to the original data. 
rand_origination

In [ ]:
rand_origination = pd.merge(rand_origination, origination_data, on =['VINTAGE','LOAN_SEQUENCE_NUMBER'], how = 'inner')
rand_origination

In [ ]:
# Now that we have the origination data, we can parse the monthly performance data file

def parse_performance(filename):
    data = []
    with open(filename, 'r') as file:
        for line in file:
            # Remove spaces and split by '|'
            processed_line = ''.join(line.split()).split('|')
            if len(processed_line) == 32:
                data.append(processed_line)
    
    # Create a DataFrame with 5 columns
    df = pd.DataFrame(data,columns=['LOAN_SEQUENCE_NUMBER', 'MONTHLY_REPORTING_PERIOD', 'CURRENT_ACTUAL_UPB', 'CURRENT_LOAN_DELINQUENCY_STATUS', 'LOAN_AGE',
                                    'REMAINING_MONTHS_TO_LEGAL_MATURITY', 'DEFECT_SETTLEMENT_DATE', 'MODIFICATION_FLAG', 'ZERO_BALANCE_CODE',
                                    'ZERO_BALANCE_EFFECTIVE_DATE', 'CURRENT_INTEREST_RATE', 'CURRENT_NON-INTEREST_BEARING_UPB', 'DUE_DATE_OF_LAST_PAID_INSTALLMENT', 
                                    'MI_RECOVERIES', 'NET_SALE_PROCEEDS', 'NON_MI_RECOVERIES', 'TOTAL_EXPENSES', 'LEGAL_COSTS', 'MAINTENANCE_AND_PRESERVATION_COSTS',
                                    'TAXES_AND_INSURANCE', 'MISCELLANEOUS_EXPENSES', 'ACTUAL_LOSS_CALCULATION', 'CUMULATIVE_MODIFICATION_COST', 'STEP_MODIFICATION_FLAG', 
                                    'PAYMENT_DEFERRAL', 'ESTIMATED_LOAN_TO_VALUE', 'ZERO_BALANCE_REMOVAL', 'DELINQUENT_ACCRUED_INTEREST',
                                    'DELINQUENCY_DUE_TO_DISASTER', 'BORROWER_ASSISTANCE_STATUS_CODE', 'CURRENT_MONTH_MODIFICATION_COST', 'INTEREST_BEARING_UPB']
                                    )
    df['VINTAGE'] = str(year) + df['LOAN_SEQUENCE_NUMBER'].str[3:5]
    return df

In [ ]:
# Replace with your file path
performance_data = pd.DataFrame([])
for year in range(min_year, max_year+1):
    filename = f'{folder_path}/sample_{year}/sample_svcg_{year}.txt'
    a = pd.merge(rand_origination, parse_performance(filename), on =['LOAN_SEQUENCE_NUMBER'], how = 'inner')
    performance_data = pd.concat([performance_data, a], ignore_index=True)
    print(f'Parsing the loan data in year {year} is done.')

print('Finished.')
performance_data.head()

In [ ]:
# For convinience, save the data in csv so that we can work with it without changing the original data. 
performance_data.to_csv('loan_performance.csv', index = False)

In [ ]:
performance_data = pd.read_csv('loan_performance.csv')

In [ ]:
print('Below shows the count of each status')
print(performance_data['ZERO_BALANCE_CODE'].value_counts())
print('---------------------------------')
print('total number of unique loan is', len(performance_data['LOAN_SEQUENCE_NUMBER'].unique()))
print('there is about this many missing zero balance status:',
      len(performance_data['LOAN_SEQUENCE_NUMBER'].unique()) - sum(performance_data['ZERO_BALANCE_CODE'].value_counts()[1:]))

In [ ]:
# If Zero balance code is 1 (prepaid or matured) and the date the balance became zero is before the maturity, the loan is fully prepaid.
condition1 = (performance_data['ZERO_BALANCE_CODE'] ==  1) & (performance_data['ZERO_BALANCE_EFFECTIVE_DATE'] < performance_data['MATURITY_DATE'])
# If zero balance code is 1 and zero balance effective date is the same day as maturity, then loan must be matured. 
condition2 = (performance_data['ZERO_BALANCE_CODE'] == 1) & (performance_data['ZERO_BALANCE_EFFECTIVE_DATE'] == performance_data['MATURITY_DATE'])
# If zero balance code is 3, 2, 96, or 9, it is a default. 
condition3 = performance_data['ZERO_BALANCE_CODE'].isin([9, 96, 3, 2])
# If zero balance code is none of above, we call loan is censored. 
condition4 = performance_data['ZERO_BALANCE_CODE'].isin([15,16])

conditions = [condition1, condition2, condition3, condition4]

values = ['Full Prepayment', 'Matured', 'Default', 'Censored']

# Create the new column using np.select
performance_data['Event_status'] = np.select(conditions, values)
performance_data['Prepayment_status'] = np.select([(performance_data['Event_status']=='Full Prepayment')], [1])
performance_data['Default_status'] = np.select([(performance_data['Event_status'] == 'Default')], [1])

In [ ]:
print('Below shows the count of each status')
print(performance_data['Event_status'].value_counts())
print(performance_data['Prepayment_status'].value_counts())
print(performance_data['Default_status'].value_counts())

In [ ]:
# Create a start and stop time for survival analysis
performance_data['Start_time'] = performance_data['LOAN_AGE']
performance_data['Stop_time'] = performance_data['LOAN_AGE'] + 1

In [ ]:
print(min(performance_data['Start_time'].unique()))
print(max(performance_data['Start_time'].unique()))

In [ ]:
# Stratify the loan term
performance_data['ORIGINAL_LOAN_TERM'].value_counts()

In [ ]:
condition1 = performance_data['ORIGINAL_LOAN_TERM'].isin(range(0,240))
condition2 = performance_data['ORIGINAL_LOAN_TERM'].isin(range(240,301))
condition3 = performance_data['ORIGINAL_LOAN_TERM'] > 300

conditions = [condition1, condition2, condition3]

values = ['0-19yr', '20-25yr', '+25yr']

performance_data['TERM'] = np.select(conditions, values)

In [ ]:
performance_data['TERM'].value_counts()

In [ ]:
performance_data['CREDIT_SCORE'].replace(9999, np.nan, inplace=True)

# Define the bins and labels for categorical scores
bins = [300, 580, 670, 740, 800, 850]
labels = ['Poor', 'Fair', 'Good', 'Very Good', 'Excellent']

# Create a new column 'credit_score_category' with the categorical scores
performance_data['SCORE'] = pd.cut(performance_data['CREDIT_SCORE'], bins=bins, labels=labels, right=False, include_lowest=True)

In [ ]:
performance_data['SCORE'].value_counts()

In [ ]:
bins = [30000, 60000, 90000, 120000, 150000, 180000, 210000, 240000, 270000, 300000, 500000, 1000000]
labels = ['3-60K', '6-90K', '9-120K', '12-150K', '15-180K', '18-210K', '21-240K', '24-270K', '27-300K', '3-500K', '500k-1M']

# Create a new column 'credit_score_category' with the categorical scores
performance_data['SIZE'] = pd.cut(performance_data['ORIGINAL_UPB'], bins=bins, labels=labels, right=False, include_lowest=True)

In [ ]:
performance_data['SIZE'].value_counts()

In [ ]:
performance_data['ORIGINAL_LOAN-TO-VALUE'].replace(999, np.nan, inplace=True)

bins = [20, 40, 60, 80, 100, 120, 150, 200, 500]
labels = ['20-40%', '40-60%', '60-80%', '80-100%', '100-120%', '120-150%', '150-200%', '200-500%']

performance_data['LTV'] = pd.cut(performance_data['ORIGINAL_LOAN-TO-VALUE'], bins=bins, labels=labels, right=False, include_lowest=True)

In [ ]:
performance_data['LTV'].value_counts()

In [ ]:
print(performance_data['AMORTIZATION_TYPE'].value_counts())
# All of the loans are fixed rate.

In [ ]:
# calculate the average of current interest rate for each month in a given year. 
monthly_avg_interest_rate = performance_data.groupby(['MONTHLY_REPORTING_PERIOD','SCORE'])['CURRENT_INTEREST_RATE'].mean().reset_index()
monthly_avg_interest_rate['CURRENT_INTEREST_RATE'].fillna(performance_data['CURRENT_INTEREST_RATE'].mean(),inplace=True)
monthly_avg_interest_rate.rename(columns={'CURRENT_INTEREST_RATE': 'Avg_Curr_Interest_Rate'}, inplace=True)


performance_data = pd.merge(performance_data, monthly_avg_interest_rate, on = ['MONTHLY_REPORTING_PERIOD', 'SCORE'], how = 'left')
performance_data['ORIGINAL_INTEREST_RATE'].fillna(performance_data['ORIGINAL_INTEREST_RATE'].mean(),inplace=True)

performance_data['Rate_gap'] = performance_data['Avg_Curr_Interest_Rate'] - performance_data['ORIGINAL_INTEREST_RATE']

In [ ]:
print(sum(performance_data['ORIGINAL_INTEREST_RATE'].isna()))
print(sum(performance_data['Rate_gap'].isna()))

In [ ]:
print(max(performance_data['Rate_gap']))
print(min(performance_data['Rate_gap']))

In [ ]:
performance_data['ORIGINAL_LOAN-TO-VALUE'].replace(999, np.nan, inplace=True)

bins = [-6, -2, 0, 1, 2, 3]
labels = ['-6-2%', '-2-0%', '0-1%', '1-2%', '2-3%']

performance_data['Rate_incv'] = pd.cut(performance_data['Rate_gap'], bins=bins, labels=labels, right=False, include_lowest=True)

In [ ]:
performance_data['Rate_incv'].value_counts()

In [ ]:
# Make a prepayment rate
# If Zero balance code is 1 (prepaid or matured) and the date the balance became zero is before the maturity, the loan is fully prepaid.
condition1 = (performance_data['ZERO_BALANCE_CODE'] ==  1) & (performance_data['ZERO_BALANCE_EFFECTIVE_DATE'] < performance_data['MATURITY_DATE'])
prepaynum = performance_data[condition1].groupby('MONTHLY_REPORTING_PERIOD')['Prepayment_status'].sum().reset_index(name='Prepayment_Count')

# THis is a condition for default
condition2 = performance_data['ZERO_BALANCE_CODE'].isin([9, 96, 3, 2])
defaultnum = performance_data[condition2].groupby('MONTHLY_REPORTING_PERIOD')['Default_status'].sum().reset_index(name='Default_Count')

# number of total active loans
totalnum = performance_data.groupby('MONTHLY_REPORTING_PERIOD')['LOAN_SEQUENCE_NUMBER'].size().reset_index(name='Loan_count')

# average rate gap, average loan size, average loan term, average credit score, average loan age, average LTV.
averages = performance_data.groupby('MONTHLY_REPORTING_PERIOD').agg({
    'Rate_gap': 'mean',
    'ORIGINAL_UPB': 'mean',
    'ORIGINAL_LOAN_TERM': 'mean',
    'CREDIT_SCORE': 'mean',
    'ORIGINAL_LOAN_TERM': 'mean',
    'ORIGINAL_LOAN-TO-VALUE': 'mean',
    'ORIGINAL_DEBT-TO-INCOME': 'mean',
    'LOAN_AGE': 'mean'
}).reset_index()

# Merge the results
totalnum = pd.merge(totalnum, averages, on='MONTHLY_REPORTING_PERIOD', how='outer')

final_data = pd.merge(totalnum, defaultnum, on='MONTHLY_REPORTING_PERIOD', how='outer')

# Merge the result with prepaynum
final_data = pd.merge(final_data, prepaynum, on='MONTHLY_REPORTING_PERIOD', how='outer')

# Fill missing values with 0
final_data.fillna(0, inplace=True)

final_data['Default_rate'] = 100*final_data['Default_Count'] / final_data['Loan_count']
final_data['Prepay_rate'] = 100*final_data['Prepayment_Count'] / final_data['Loan_count']
final_data

In [ ]:
performance_data.to_csv('LOAN_surv.csv', index=False)
final_data.to_csv('pp_df_rate.csv', index = False)